# Equispaced Grid

### Code to create a csv file with an esquispaced grid  500m x 500 in [file]_interpolated_filled_500m.csv

In [1]:
# Importing libraries
import pandas as pd
import pyproj
import numpy as np
from scipy.interpolate import griddata

In [2]:
# Read the csv file with the velocities reported by White et al. 2019
df = pd.read_csv("/Users/cecilia/PYEIFMM/White_et_al_2021__Ridgecrest_2019_velocity.csv")

# Define the input coordinate system (latitude and longitude)
input_crs = pyproj.CRS("EPSG:4326")  # EPSG:4326 represents WGS 84 (latitude and longitude)

# Define the output coordinate system (latitude and longitude)
output_crs = pyproj.CRS("EPSG:26711")  # EPSG:26711 represents UTM Zone 11 of the north hemisphere NAD27)

# Create a coordinates transformer
transformer = pyproj.Transformer.from_crs(input_crs, output_crs, always_xy=True)

# Apply the transformation to the latitude and longitude columns
df['x'], df['y'] = transformer.transform(df['longitude'].values, df['latitude'].values)

# Minimun latitude (y) and longitude (x) values
min_proyx=df['x'].min()
min_proyy=df['y'].min()

# Adjust the minimun value of x and y to (0,0) 
df['x'] -= df['x'].min()
df['y'] -= df['y'].min()
df['z'] = df['depth']*1000   # Convert depth from kilometers to meters 

# Save the new DataFrame in a new csv file
df.to_csv("/Users/cecilia/PYEIFMM/White_et_al_2021__Ridgecrest_2019_velocity_transformed.csv", index=False)

In [3]:
# Input file information
min_long=df['longitude'].min()
max_long=df['longitude'].max()
min_lat=df['latitude'].min()
max_lat=df['latitude'].max()
min_z=df['z'].min()
crs_geo="EPSG:4326"

# Output file information
min_x=df['x'].min()
max_x=df['x'].max()
min_y=df['y'].min()
max_y=df['y'].max()
crs_proy="EPSG:26711"

In [4]:
# Show some input file values
print(min_long,
max_long,
min_lat,
max_lat,
min_z)

-119.0 -116.0 34.5 37.0 -3000.0


In [5]:
# Show some output file values
print(min_x,
max_x,
min_y,
max_y,
min_z)

print(min_proyx, min_proyy)

0.0 275423.97344550805 0.0 279143.87236170284 -3000.0
316459.9297286207 3817400.650678637


In [6]:
# Relevant coordinates: Station and test events A, B, C

# STATION COORDINATES
# Original coordinates in EPSG:4326 (latitude y longitude)
sta_latitud = 35.81574
sta_longitud = -117.59751

# Transform the station coordinates to EPSG:26711
sta_x, sta_y = transformer.transform(sta_longitud, sta_latitud)

# Adjust the coordinates to the origin
sta_x -= min_proyx
sta_y -= min_proyy

print("Transformed station coordinates:")
print("X:", sta_x)
print("Y:", sta_y)

# KEEP USING THIS CELL TO ADD THE TEST EVENTS CORDINATES...

Transformed station coordinates:
X: 129642.07044546341
Y: 146078.6042745374


In [7]:
# Build the equispaced grid

# Read the velocity transformed csv file
df = pd.read_csv("/Users/cecilia/PYEIFMM/White_et_al_2021__Ridgecrest_2019_velocity_transformed.csv")

# Get a list of unique "depth" values
depth_values = df['depth'].unique()

# Create a new DataFrame to store the interpolated results in the new grid
df_interpolated_new = pd.DataFrame()

list_array_vp=[]
list_array_vs=[]

# Iterate through each depth layer and perform interpolation
for depth_value in depth_values:
    # Filter the DataFrame for the current depth layer
    df_depth = df[df['depth'] == depth_value]

    # Create a 500 meter equispaced grid in x and y for this layer
    x_min, x_max = df_depth['x'].min(), df_depth['x'].max()
    y_min, y_max = df_depth['y'].min(), df_depth['y'].max()
    
    x_grid_new = np.arange(x_min, x_max + 500, 500)
    y_grid_new = np.arange(y_min, y_max + 500, 500)

    x_mesh_new, y_mesh_new = np.meshgrid(x_grid_new, y_grid_new)
    x_mesh_new=np.transpose(x_mesh_new)
    y_mesh_new=np.transpose(y_mesh_new)

    # Interpolate Vp and Vs values ​​on the new 2D grid with linear extrapolation
    vp_interpolated = griddata((df_depth['x'], df_depth['y']), df_depth['Vp'], (x_mesh_new, y_mesh_new), method='linear', fill_value=np.nan)
    vs_interpolated = griddata((df_depth['x'], df_depth['y']), df_depth['Vs'], (x_mesh_new, y_mesh_new), method='linear', fill_value=np.nan)

    # Calculate the average Vp and Vs within this depth layer
    average_vp = df_depth['Vp'].mean()
    average_vs = df_depth['Vs'].mean()

    # Replace the NaN values ​​in "Vp" and "Vs" with the averages calculated of this depth layer
    vp_interpolated[np.isnan(vp_interpolated)] = average_vp
    vs_interpolated[np.isnan(vs_interpolated)] = average_vs

    list_array_vp.append(vp_interpolated)
    list_array_vs.append(vs_interpolated)
    
    # Create a DataFrame for this interpolated depth layer in the new grid
    df_depth_interpolated = pd.DataFrame({
        'x': x_mesh_new.flatten(),
        'y': y_mesh_new.flatten(),
        'z' : depth_value * 1000 , 
        'Vp': vp_interpolated.flatten(),
        'Vs': vs_interpolated.flatten(),
        'depth': depth_value  # Add the "depth" column with the current depth value
    })

    # Add the results of this depth layer to the main DataFrame of the new grid
    df_interpolated_new = pd.concat([df_interpolated_new, df_depth_interpolated], ignore_index=True)

# Save the resulting DataFrame to a new CSV file with the new grid
df_interpolated_new.to_csv("/Users/cecilia/PYEIFMM/White_et_al_2021__Ridgecrest_2019_velocity_interpolated_filled_500m.csv", index=False)


In [8]:
# Stack bidimensional velocities along the "depth" axis
vp_s = np.dstack(list_array_vp)
vs_s = np.dstack(list_array_vs)

# Convert the resulting tridimensional velocities array to float32 data type
vp3 = vp_s.astype(np.float32)
vs3 = vs_s.astype(np.float32)

In [9]:
# Definition of file paths to save the tridimensional arrays
ruta_vp = "/Users/cecilia/PYEIFMM/vp_s.npy"
ruta_vs = "/Users/cecilia/PYEIFMM/vs_s.npy"

In [10]:
# Save velocities arrays in .npy files
np.save(ruta_vp, vp3)
np.save(ruta_vs, vs3)

In [11]:
vp3.dtype

dtype('float32')

### Store all used parameters in a .json file

In [12]:
z_values = df_interpolated_new['z'].unique()
x_values = df_interpolated_new['x'].unique()
y_values = df_interpolated_new['y'].unique()

In [13]:
# Define .json file parameters
import json
parameters = {
    "min_long": min_long,
    "max_long": max_long,
    "min_lat": min_lat,
    "max_lat": max_lat,
    "crs_geo": crs_geo,
    "crs_proy": crs_proy,
    "min_x": min_proyx,
    "min_y": min_proyy,
    "min_z" : min_z,
    "nrx": len(x_values),
    "nry": len(y_values),
    "nrz": len(z_values)
}

# Define the .json file name
file_name = '/Users/cecilia/PYEIFMM/proyections.json'

# Saving the parameters in the .json file
with open(file_name, 'w') as file:
    json.dump(parameters, file, indent=4)

In [14]:
vp3.shape

(552, 560, 87)

In [15]:
parameters

{'min_long': -119.0,
 'max_long': -116.0,
 'min_lat': 34.5,
 'max_lat': 37.0,
 'crs_geo': 'EPSG:4326',
 'crs_proy': 'EPSG:26711',
 'min_x': 316459.9297286207,
 'min_y': 3817400.650678637,
 'min_z': -3000.0,
 'nrx': 552,
 'nry': 560,
 'nrz': 87}

In [16]:
# Just visualizing some data values
list_array_vp[0].shape

(552, 560)

In [17]:
vp_interpolated.shape

(552, 560)

In [18]:
len(df_depth['x'])

75551

In [19]:
print((x_max/500))

550.8479468910161


In [20]:
xa_mesh_new, ya_mesh_new = np.meshgrid(x_grid_new, y_grid_new)

In [21]:
ya_mesh_new.shape

(560, 552)